In [7]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pandas import Series, DataFrame, Panel
import matplotlib.pyplot as plt
from matplotlib import collections as mc 
import os

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Tunable Variables
days_to_process = 7
pd.options.display.max_rows = 1000

# Loading data
col_names = ['TrackID','FrameNo', 'X', 'Y']
# day1 = pd.read_table("001_20130101.txt",delimiter=" ", header=None, names=col_names)
# day2 = pd.read_table("001_20130102.txt",delimiter=" ", header=None, names=col_names)
# day3 = pd.read_table("001_20130103.txt",delimiter=" ", header=None, names=col_names)
# day4 = pd.read_table("001_20130104.txt",delimiter=" ", header=None, names=col_names)
# day5 = pd.read_table("001_20130105.txt",delimiter=" ", header=None, names=col_names)
# day6 = pd.read_table("001_20130106.txt",delimiter=" ", header=None, names=col_names)
# day7 = pd.read_table("001_20130107.txt",delimiter=" ", header=None, names=col_names)

day1 = pd.read_table("001_20130212.txt",delimiter=" ", header=None, names=col_names)
day2 = pd.read_table("001_20130214.txt",delimiter=" ", header=None, names=col_names)
day3 = pd.read_table("001_20130215.txt",delimiter=" ", header=None, names=col_names)
day4 = pd.read_table("001_20130216.txt",delimiter=" ", header=None, names=col_names)
day5 = pd.read_table("001_20130217.txt",delimiter=" ", header=None, names=col_names)
day6 = pd.read_table("001_20130218.txt",delimiter=" ", header=None, names=col_names)
day7 = pd.read_table("001_20130219.txt",delimiter=" ", header=None, names=col_names)

frames = [day1, day2, day3, day4, day5, day6, day7]
concatDay = pd.concat(frames, ignore_index=True)
# concatDay = concatDay.sort_values(by='TrackID', ascending=True)
concatDay



,TrackID,FrameNo,X,Y
0,113,169,534,369
1,113,170,531,364
2,113,171,530,364
3,113,172,527,366
4,113,173,525,366
5,113,174,520,368
6,113,175,518,368
7,113,176,516,369
8,113,177,515,369
9,113,178,511,371


In [5]:
# Prepare .tra extension file for TraClus clustering algorithm
outputFile = []
concatDay = concatDay[['TrackID', 'X', 'Y']]
concatDayWithoutID = concatDay[['X', 'Y']]
concatDayWithID = concatDay[['TrackID']]
concatDayWithID = concatDayWithID.drop_duplicates('TrackID')
concatDayWithID = concatDayWithID.values  #it's an array now


# slicing the dataframe based on ID
row_iterator = concatDay.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []

####################suspected bug################################
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']
        
# print(slicingArray)
newList = np.split( concatDayWithoutID, slicingArray )

####################suspected bug################################

#Now: We need to combine every single piece of data to conform with .tra
#header: number of dimensions
#      : number of trajectories
number_of_dimension = 2
number_of_trajectory = len(concatDayWithID)
arrayForHeader = str(number_of_dimension) + '\n' + str(number_of_trajectory) + '\n'
print(concatDayWithID)
#format: TrackID no_trajectory_point X1 Y1 X2 Y2... Xn Yn
arrayForXY = []
number_of_ID = len(concatDayWithID)
counter_for_points = 0
counter_for_ID = 0
# Probably need to create another outer loop for diff sets of newList, like newList[1], newList[2]

while counter_for_ID < number_of_ID:
    # reset
    counter_for_points = 0
    # Convert ID to proper string form
    tempID = concatDayWithID[counter_for_ID]
    number_of_trajectory_points = len(newList[counter_for_ID])
#     print(number_of_trajectory_points)
    stringForTempID = str(tempID)
    stringForTempID = stringForTempID[1:-1]
    # concatenation
    if len(arrayForXY) == 0:
        arrayForXY = stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    else:
        arrayForXY += stringForTempID + ' ' + str(number_of_trajectory_points) + ' '
    while counter_for_points < number_of_trajectory_points:  
        # Select X and Y
        tempHelper = newList[counter_for_ID]
        tempArray = tempHelper.iloc[counter_for_points]
        helperX = tempArray[0]
        helperY = tempArray[1]
        arrayForXY += str(helperX) + ' ' + str(helperY) + ' '
        counter_for_points += 1
        if counter_for_points == number_of_trajectory_points:
            arrayForXY += '\n'
    counter_for_ID += 1

finalOutput = arrayForHeader + arrayForXY    

# print(newList[0])
# print(tempArray)
# print(arrayForHeader)
# print(arrayForXY)
# print(len(arrayForXY))
# print((concatDayWithID))
# print(finalOutput)

output = open('LOST3.tra', 'w')
output.write(finalOutput)
output.close()
# print(newList)



[[  113]
 [  584]
 [  661]
 [  987]
 [ 1934]
 [ 2076]
 [ 2592]
 [ 3085]
 [ 3315]
 [ 4276]
 [ 6116]
 [ 7200]
 [ 9308]
 [ 9326]
 [ 9861]
 [10063]
 [11111]
 [13808]
 [17172]
 [17253]
 [17384]
 [17931]
 [18621]
 [19031]
 [19103]
 [19247]
 [19257]
 [19293]
 [19521]
 [19579]
 [20217]
 [20287]
 [20392]
 [20818]
 [22223]
 [23019]
 [23732]
 [24145]
 [24686]
 [25078]
 [25411]
 [25713]
 [26084]
 [26187]
 [26578]
 [26686]
 [26809]
 [26940]
 [26948]
 [29409]
 [29476]
 [29942]
 [30342]
 [30718]
 [30792]
 [31969]
 [32335]
 [32594]
 [32639]
 [33289]
 [34297]
 [  172]
 [  206]
 [  658]
 [  683]
 [ 1002]
 [ 1038]
 [ 1042]
 [ 1057]
 [ 1130]
 [ 1197]
 [ 1304]
 [ 1364]
 [ 1367]
 [ 1384]
 [ 1522]
 [ 1567]
 [ 1772]
 [ 1880]
 [ 1924]
 [ 2027]
 [ 2037]
 [ 2090]
 [ 2228]
 [ 2319]
 [ 2330]
 [ 2479]
 [ 2781]
 [ 2833]
 [ 2941]
 [ 3013]
 [ 3030]
 [ 3037]
 [ 3130]
 [ 3147]
 [ 3175]
 [ 3263]
 [ 3314]
 [ 3350]
 [ 3647]
 [ 3669]
 [ 3749]
 [ 3786]
 [ 3811]
 [ 3891]
 [ 3952]
 [ 3985]
 [ 4036]
 [ 4089]
 [ 4105]
 [ 4168]
 

In [8]:
# Get the output from TraClus, revamp and prepare data to visualize
# to achieve this, i need to read from text file and reconstruct it back to a dataframe.
fileOpener = open('LOST3output.txt', 'r')
lines_ori = fileOpener.readlines()
# print(type(lines))

########Choose the rows with X Y coordinates####################################
#a loop is needed to differentiate the xy and affilitated trajectory's id
counterLineNumber = 0 
for lines in lines_ori:
    indexOfTraj = lines.find("trajectoryID: 0")
    if indexOfTraj == 0:
        break
    counterLineNumber += 1

print(counterLineNumber)
start = 2
end = counterLineNumber 
step = 2
lines = lines_ori[start:end:step]
# print(lines)
################################################################################

####################Choose the rows with ID and number of points################
clusterID_array = []
num_points = []
start = 1
end = counterLineNumber
step = 2
lines_ID = lines_ori[start:end:step]
length_lines_ID = len(lines_ID)
counter_for_linesID = 0
# print(lines_ID[0])

while counter_for_linesID < length_lines_ID:
    temp_lines_ID = str(lines_ID[counter_for_linesID])
    temp_index = temp_lines_ID.find(" ")
    temp_index3 = temp_lines_ID.find("P")
    temp_index2 = temp_lines_ID.find("  ", 14, len(temp_lines_ID))
    clusterID_array += [temp_lines_ID[temp_index + 1:temp_index3]]
    num_points += [temp_lines_ID[temp_index2 + 2:]]
    counter_for_linesID += 1
#     print(clusterID_array)
##################################################################################

#Strip the string into appropriate form
element = str(lines)
elementText = element[0:-1]
elementText = str.replace(elementText, '\\' , '')
elementText = str.replace(elementText, "n", '')
elementText = str.replace(elementText, "   ", " ")
elementText = str.replace(elementText, ",", "\n")
elementText = str.replace(elementText, "\n", '')
elementText = str.replace(elementText, "'", "" )
elementText = str.replace(elementText, "  "," ")
elementText = str.replace(elementText, "[", "")
# print(type(elementText))
# print(elementText)

#put in 2d array form and convert to integer
numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
numpyInt = numpyString.astype(int)
number_of_rows = int( len(numpyInt)/2 )
numpyInt = numpyInt.reshape((number_of_rows,2))

#convert numpy array to dataframe
dataframeInt = pd.DataFrame(numpyInt)
dataframeInt.columns = ['X', 'Y']
dataframeInt = dataframeInt.convert_objects(convert_numeric=True)

#Assign ID to specific tracks
temp_array_ID = []
total_num_points = len(num_points)
counter_for_total_num = 0
while counter_for_total_num < total_num_points:
    tempCounter_for_num = 0
    while tempCounter_for_num < int(num_points[counter_for_total_num]):
        temp_array_ID += [clusterID_array[counter_for_total_num]]
        tempCounter_for_num += 1
    counter_for_total_num += 1

#finalized output
# print(num_points)
dataframeInt['TrackID'] = temp_array_ID
cols = dataframeInt.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataframeInt = dataframeInt[cols]
    
# print(numpyInt)
# print(dataframeInt.dtypes)
print(dataframeInt)
# print(type(numpyInt))

# dummyTest2 = list(numpyInt)
# dummyTest2 = dataframeInt
# print(dummyTest2)

9
   TrackID    X    Y
0      0    152  248
1      0    188  251
2      0    225  254
3      0    262  254
4      0    298  256
5      0    334  259
6      0    371  261
7      0    407  263
8      0    445  266
9      0    481  269
10     1    307  301
11     1    348  319
12     1    380  337
13     1    413  354
14     1    449  369
15     1    482  383
16     1    516  400
17     1    550  416
18     1    581  434
19     1    619  453
20     2    538  429
21     2    500  439
22     2    466  449
23     2    430  457
24     2    393  464
25     2    355  470
26     3    186  264
27     3    149  274
28     3    115  290
29     3     80  305
30     3     47  322
31     3     14  337


C:\Users\Ice\Anaconda3\lib\site-packages\ipykernel\__main__.py:69: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [31]:
#Decide the color for each cluster
##########stripping the trajectory id#################
startTraj_ID = end + 1
endTraj_ID = len(lines_ori)
stepTraj_ID = 2
traj_ID = lines_ori[startTraj_ID:endTraj_ID:stepTraj_ID]
counter_cluster_ID = 0
array_for_cluster_ID = []
count_for_cluster_ID = []
## Form ***clusterID***
# print(traj_ID[0])
while counter_cluster_ID < len(traj_ID):
    count_cluster_ID = 0
    number_of_string = traj_ID[counter_cluster_ID].count(' ')
    while count_cluster_ID < number_of_string:
        array_for_cluster_ID += [counter_cluster_ID]
        count_cluster_ID += 1
    counter_cluster_ID += 1
array_for_cluster_ID = pd.DataFrame(array_for_cluster_ID)
# print((array_for_cluster_ID))

#sample: elementText = str.replace(elementText, '\\' , '')
#sample: numpyString = np.fromstring(elementText, dtype=float, sep= ' ' or '\n')
#sample" dataframeInt = pd.DataFrame(numpyInt)
## Stripping string to form ***trajectoryID***
traj_ID = str(traj_ID)
traj_ID = str.replace(traj_ID, '[\'', '')
traj_ID = str.replace(traj_ID, '\']', '')
traj_ID = str.replace(traj_ID, ', ', '\n')
traj_ID = str.replace(traj_ID, '\\n', '')
traj_ID = str.replace(traj_ID, '\'', '')
traj_ID = np.fromstring(traj_ID, dtype=int, sep=' ')
traj_ID = pd.DataFrame(traj_ID)
traj_ID.columns = ['TrackID']
traj_ID['ClusterID'] = array_for_cluster_ID
# print(traj_ID)
######################################################

#check the matched value through joining
tempDF = pd.merge(concatDay, traj_ID, on='TrackID')
tempDF = tempDF[['TrackID', 'FrameNo', 'X', 'Y', 'ClusterID_y']]
tempDF = tempDF.rename(columns={"ClusterID_y":"ClusterID"})
temp_traj_ID = tempDF
temp_traj_ID = temp_traj_ID[['TrackID', 'ClusterID']]

# temp_traj_ID = tempDF.join(concatDay, on='TrackID', lsuffix='_l', rsuffix='_r')


tempDF2 = tempDF[['TrackID', 'FrameNo','X', 'Y']]
# tempDF3 = tempDF[['ClusterID']]
tempDF4 = concatDay
tempDF4['ClusterID'] = np.nan
# tempDF4['ClusterID_'] = np.nan
# df1[(~df1.col1.isin(common.col1))&(~df1.col2.isin(common.col2))] format
# tempDF4 = tempDF4[(~tempDF.TrackID.isin(tempDF.TrackID))]


#####maybe i should try looping through dataframe although it's more tiring
# cluster_id_for_concatDay = []
# for index, row in concatDay.iterrows():
#     temp_row_concatDay = row['TrackID']
#     for index, row in traj_ID.iterrows():
#         temp_row_temp_traj_ID = row['TrackID']
#         if temp_row_concatDay == temp_row_temp_traj_ID:
#             cluster_id_for_concatDay += row['ClusterID']
#         else:
#             cluster_id_for_concatDay += [1000]
# tempDF4 = pd.concat([tempDF4, temp_traj_ID], axis=1, join_axes=[temp_traj_ID.TrackID])       
########################Assign Color##################
tempColor = np.where(tempDF['ClusterID'] == 0, 'blue', 
                     np.where( tempDF['ClusterID'] == 1, 'yellow', 
                              np.where(tempDF['ClusterID'] == 2, 'cyan',
                                       np.where(tempDF['ClusterID'] == 3, 'magenta','black'))))                                       
######################################################
print(len(tempDF4))
print(len(tempDF))
# print(tempDF)
print(len(tempColor))
# print(cluster_id_for_concatDay)
# print(temp_traj_ID)
print(tempDF4)


19115
12434
12434
       TrackID  FrameNo    X    Y  ClusterID
0          113      169  534  369        NaN
1          113      170  531  364        NaN
2          113      171  530  364        NaN
3          113      172  527  366        NaN
4          113      173  525  366        NaN
5          113      174  520  368        NaN
6          113      175  518  368        NaN
7          113      176  516  369        NaN
8          113      177  515  369        NaN
9          113      178  511  371        NaN
10         113      179  509  373        NaN
11         113      180  506  373        NaN
12         113      181  503  373        NaN
13         113      182  500  374        NaN
14         113      183  498  376        NaN
15         113      184  495  376        NaN
16         113      185  493  376        NaN
17         113      186  491  378        NaN
18         113      187  487  380        NaN
19         113      188  486  381        NaN
20         113      189  483  382    

In [9]:
# Construct lines for visualization in plot
# day1 = day1[['TrackID','X','Y']]
# dfPlotDay1 = day1[['X', 'Y']]
#assignment without colors for all days
# dataframeInt = concatDay
#assignment with colors for all days
dataframeInt = tempDF2
#if clustered, then just comment out both lines above

concatenator = dataframeInt[['TrackID','X','Y']]
concatenator2 = dataframeInt[['X', 'Y']]

row_iterator = concatenator.iterrows()
row_id, this_row = next(row_iterator)
#slicing dataframe to respective arrays for LineCollection function
initialID = this_row['TrackID']
slicingArray = []
for i, rows in row_iterator:
    if rows['TrackID'] != initialID:
        slicingArray.append(i)
        initialID = rows['TrackID']

testChamp = np.split( concatenator2, slicingArray )
testChampSize = len(testChamp)

###commented because not workable####
# indices = [0, 1]
# abc = np.take(testChamp[1], indices, axis = 1)
# abc = abc.as_matrix()
#####################################
dummyTest2 = list(testChamp)

# print(len(dummyTest2))
# print(concatDay)

In [10]:
#Clustering lines(Kmeans)
# indices = [1, 2]
indices = [0, 1]
counterForLineConstruction = 0

#line preparation
linesegment = mc.LineCollection(dummyTest2, linewidths = 2, linestyles='solid', colors=tempColor)
# linesegment = mc.LineCollection(dataframeInt, linewidths = 2, linestyles='solid', colors='black')

#canvas setup
x = np.arange(641)
ys = x[0:480, np.newaxis]

# set plot limits
ax = plt.axes()
ax.set_xlim(x.min(), x.max())
ax.set_ylim(ys.min(), ys.max())
ax.add_collection(linesegment)
plt.gca().invert_yaxis()
im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)
plt.show()

In [14]:
#Remake data for clustering purposes
#Clustering tracks for day1
day1 = day1[['X','Y']]
day2 = day2[['X','Y']]
day3 = day3[['X','Y']]
day4 = day4[['X','Y']]
day5 = day5[['X','Y']]
day6 = day6[['X','Y']]
day7 = day7[['X','Y']]

# concatDay = [day1, day2, day3, day4, day5, day6, day7]
concatDayTrain = [day1, day2, day3, day4, day5]
concatDayTrain = pd.concat(concatDayTrain)
concatDayTrain

concatDayTest = [day6, day7]
concatDayTest = pd.concat(concatDayTest)
concatDayTest

kmeans = KMeans(n_clusters=8, random_state=0).fit(concatDayTrain)
kmeansTrans = kmeans.transform(concatDayTrain)
centroids = kmeans.cluster_centers_
distance = kmeans.inertia_
labels = kmeans.labels_    #indication for each datapoint that determines the cluster it belongs to

tempDF = concatDayTrain
tempDF['cluster_label'] = Series(labels, index = tempDF.index)
# print(tempDF)
#color code assignment
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


centroidsDF = pd.DataFrame(data=centroids[0:,0:], columns=['X','Y'])
# plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
# plt.scatter(concatDayTrain['X'], concatDayTrain['Y'], c=tempColor, s= 50) #red color dots as training set


# print(kmeansTrans.size)
# print(concatDayTrain.size)

# prediction
prediction = kmeans.predict(concatDayTest)
tempDF = concatDayTest
tempDF['cluster_label'] = Series(prediction, index = tempDF.index)

#color code assignment for prediction
tempColor = np.where(tempDF['cluster_label'] == 0, 'black', 
                     np.where( tempDF['cluster_label'] == 1, 'yellow', 
                              np.where(tempDF['cluster_label'] == 2, 'magenta', 
                                       np.where(tempDF['cluster_label'] == 3, 'cyan', 
                                                np.where(tempDF['cluster_label'] == 4, 'red', 
                                                         np.where(tempDF['cluster_label'] == 5, 'green', 
                                                                 np.where(tempDF['cluster_label'] == 6, 'blue', 'orange'))))) ))


plt.scatter(centroidsDF['X'], centroidsDF['Y'], c='black', linewidths=50, marker='x')
plt.scatter(tempDF['X'], tempDF['Y'], c=tempColor, s=50 )
print(type(tempColor))
# plt.gca().invert_yaxis()
plt.show()

im = plt.imread('overlayingImage.png')
implot = plt.imshow(im)

        X    Y  cluster_label
0     534  369              5
1     531  364              5
2     530  364              5
3     527  366              5
4     525  366              5
5     520  368              5
6     518  368              5
7     516  369              5
8     515  369              1
9     511  371              1
10    509  373              1
11    506  373              1
12    503  373              1
13    500  374              1
14    498  376              1
15    495  376              1
16    493  376              1
17    491  378              1
18    487  380              1
19    486  381              1
20    483  382              1
21    478  382              1
22    475  384              1
23    472  385              1
24    471  385              1
25    468  385              1
26    464  388              1
27    462  389              1
28    457  390              1
29    455  390              1
30    452  392              1
31    448  393              1
32    444 